In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
file_name = repo_path / 'data/covert-search/large-field/p2_data_AZ_t1.mat'
data = loadmat(file_name)
print(data.keys())

In [ ]:
taget_amplitude = data['targetAmplitude']
target_location = data['tLocation']
spot_center = data['spotCenters']
human_response = data['hResponse']
n_location = data['nLocations'][0][0]
spot_diameter = data['spotLength'][0][0]
stimulus_design_size= data['totalLength'][0][0]
monitor_width = data['monitorPx'][0][0]
monitor_height = data['monitorPx'][0][1]

In [ ]:
shifted_spot_center = spot_center.copy()
shifted_spot_center[:,0] += (monitor_height - stimulus_design_size)//2
shifted_spot_center[:,1] += (monitor_width - stimulus_design_size)//2

accurate_response = target_location == human_response

In [ ]:
stimulus_region = np.zeros((monitor_height, monitor_width))
pixel_col, pixel_row = np.meshgrid(np.arange(monitor_width), np.arange(monitor_height))

for index_location in range(n_location-1):
    spot_region = (pixel_row - shifted_spot_center[index_location, 0])**2+(pixel_col - shifted_spot_center[index_location, 1])**2<= spot_diameter**2/4
    stimulus_region[spot_region] = 1

In [ ]:
fig, ax = plt.subplots()
ax.imshow(stimulus_region)

for index_location in range(n_location-1):
    ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{index_location+1}', ha='center', va='center')

In [ ]:
local_accuracy_region = np.empty((monitor_height, monitor_width))
pixel_col, pixel_row = np.meshgrid(np.arange(monitor_width), np.arange(monitor_height))

local_accuracy = accurate_response.mean(axis=(0,1,2))

for index_location in range(n_location-1):
    spot_region = (pixel_row - shifted_spot_center[index_location, 0])**2+(pixel_col - shifted_spot_center[index_location, 1])**2<= spot_diameter**2/4
    local_accuracy_region[spot_region] = local_accuracy[index_location]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_accuracy_region)

for index_location in range(n_location-1):
    ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{local_accuracy[index_location]:.2f}', ha='center', va='center')

In [ ]:
array_eccentral_distance = np.zeros((n_location-1,))

for index_location in range(n_location-1):
    array_eccentral_distance[index_location] = np.sqrt((spot_center[index_location, 0] - stimulus_design_size//2)**2+(spot_center[index_location, 1] - stimulus_design_size//2)**2)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(array_eccentral_distance, local_accuracy, s=250)

ax.set(xlabel='Eccentral distance', ylabel='Accuracy')
plt.show()

In [ ]:
local_accuracy_by_session = accurate_response.mean(axis=(0,2))
n_session = local_accuracy_by_session.shape[0]
array_index_location = np.arange(1,n_location)

In [ ]:
fig, ax = plt.subplots()

for index_session in range(n_session):
    ax.scatter(array_index_location, local_accuracy_by_session[index_session,:], s=250, label=f'Session {index_session+1}')

ax.legend(loc='best')
ax.set(xlabel='Spot', ylabel='Accuracy', xticks=array_index_location)
plt.show()

In [ ]:
fig, ax = plt.subplots()

for index_session in range(n_session):
    ax.scatter(array_eccentral_distance, local_accuracy_by_session[index_session,:], s=250, label=f'Session {index_session+1}')

ax.legend(loc='best')
ax.set(xlabel='Eccentral distance', ylabel='Accuracy')
plt.show()